In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%matplotlib inline
import PIL
import tqdm
import pickle
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from skimage.filters import gabor
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops

In [ ]:
import pandas as pd 
import numpy as np


In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split




import os
import shutil

import warnings 
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
import random
import pickle
from sklearn import model_selection
from pylab import *
#from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import scipy.fftpack
from skimage import data, io
from scipy.signal import convolve2d as conv2
#import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import train_test_split,cross_val_score
from skimage import color, data, restoration
from skimage.feature import greycomatrix,greycoprops

from sklearn.metrics import  accuracy_score, f1_score, precision_score,confusion_matrix, recall_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC

%matplotlib inline
from sklearn import preprocessing

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
import skimage.io as io
import skimage as sk
from skimage import measure
import math
from math import sqrt
from skimage import io, color, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy
from scipy.stats import skew, kurtosis
from numpy import asarray
from PIL import Image

import math
from math import sqrt

import skimage as sk
from skimage import measure

from numpy import asarray
from PIL import Image
import math
from math import sqrt

import skimage as sk
from skimage import measure

from numpy import asarray
from PIL import Image

import numpy as np
from skimage import io, color, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy

import scipy
import cv2
from scipy.stats import skew, kurtosis
import skimage
from skimage.filters import threshold_otsu

In [ ]:
####################### feature extraction###################




#############give directory 

dirr1= r'/content/drive/MyDrive/Research/ECG/Dataset/Segmentation/Covid 19/V6' #### source folder

csv=r'/content/drive/MyDrive/Research/ECG/Dataset/CSV/Covid19/Covid19_V6.csv' ### csv save path
### must change csv path

c=1 ### c=class 1=0 or class 2=1  

#####################################################################################################



black=0
j=0
df1 = pd.DataFrame(columns=['class', 'Area', 'PA_ratio', 'Solidity', 'Circularity', 'EquivDiameter', 'ConvexArea', 'Extent', 'FilledArea', 'Major axis length', 'Minor axis length', 'Mean', 'Standard Deviation', 'Shannon_Entropy', 'Glcm_entropy', 'Skewness', 'kurtosis','lbp_energy','lbp_entropy','gabor_energy','gabor_entropy'])

import os
onlyfiles = next(os.walk(dirr1))[2] #dir is your directory path as string
print(len(onlyfiles))



for i in os.listdir(dirr1):
    path = os.path.join(dirr1,i)
    im = io.imread(path)
    height = im.shape[0]
    width = im.shape[1]
    print('working on img ',i)
    props=sk.measure.regionprops(im)
    
    
    if len(props)!=0:
        pr=[]
        for prop in props: 
            pr.append(prop.area)
        num = np.array(pr)
        a=props[np.argmax(num)]
        
        imgcv = cv2.imread(path)
        gray = cv2.cvtColor(imgcv,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        ave = cv2.mean(thresh)[0]/255
        area = ave*height*width###########################
        try:
            image=skimage.io.imread(path,as_gray=True)
            thresh = threshold_otsu(image)
            binary = image > thresh
            perimeter=measure.perimeter(binary)################


            PA_ratio=round(perimeter/area,4)
            EquivDiameter=round(sqrt(4*area/math.pi),4)
            Circularity=round((4*area*math.pi)/(perimeter*perimeter),4)
            ConvexArea=round(a.convex_area,4)

        except:
            
            print('black found \n')
            black+=1
      
            continue    
       









        Extent=round(a.extent,4)
        Solidity=round(area/ConvexArea,4)
        FilledArea=round(a.filled_area,4)
        maj_ax_le=round(a.major_axis_length,4)
        min_ax_le=round(a.minor_axis_length,4)
        #LS_ratio=round(maj_ax_le/min_ax_le,4)
        EulerNumber=round(a.euler_number,4)



        image=Image.open(path)
        pixels=asarray(image)
        # convert from integers to floats
        pixels=pixels.astype('float32')
        # calculate global mean
        mean=round(pixels.mean(),3)

        pixels=asarray(image)
        # convert from integers to floats
        pixels=pixels.astype('float32')
        # calculate global mean and standard deviation
        std=round(pixels.std(),3)

        #print('Standard Deviation: %.3f' % (std))
        sha_entropy=round(skimage.measure.shannon_entropy(image),4)


        #LBP & Gabor
        img_gray = image.convert('L') 
        img_arr = np.array(img_gray)
        feat_lbp = local_binary_pattern(img_arr,8,1,'uniform') #Radius = 1, No. of neighbours = 8
        feat_lbp = np.uint8((feat_lbp/feat_lbp.max())*255)
        lbp_hist,_ = np.histogram(feat_lbp,8)
        lbp_hist = np.array(lbp_hist,dtype=float)
        lbp_prob = np.divide(lbp_hist,np.sum(lbp_hist))
        lbp_energy = np.sum(lbp_prob**2)
        lbp_entropy = -np.sum(np.multiply(lbp_prob,np.log2(lbp_prob)))

        gaborFilt_real,gaborFilt_imag = gabor(img_arr,frequency=0.6)
        gaborFilt = (gaborFilt_real**2+gaborFilt_imag**2)//2
        gabor_hist,_ = np.histogram(gaborFilt,8)
        gabor_hist = np.array(gabor_hist,dtype=float)
        gabor_prob = np.divide(gabor_hist,np.sum(gabor_hist))
        gabor_energy = np.sum(gabor_prob**2)
        gabor_entropy = -np.sum(np.multiply(gabor_prob,np.log2(gabor_prob)))

        


        rgbImg=io.imread(path)
        grayImg=img_as_ubyte(color.rgb2gray(rgbImg))
        distances=[1,2,3]
        angles=[0,np.pi/4,np.pi/2,3*np.pi/4]
        properties=['energy','homogeneity']
        glcm=greycomatrix(grayImg,distances=distances,angles=angles,symmetric=True,normed=True)
        glcm_entropy=round(-np.sum(glcm*np.log2(glcm+(glcm==0))),4)
        im=cv2.imread(path)
        colon_number=0
        Skewness=round(skew(im.reshape(-1,3)[:,colon_number]),4)
        Kurtosis=round(kurtosis(im.reshape(-1,3)[:,colon_number]),4)
        #df1.loc[j] =list(area)+list(PA_ratio)+list(Solidity)+list(Circularity)+list(EquivDiameter)+list(ConvexArea)+list(Extent)+list(FilledArea)+list(maj_ax_le)+list(min_ax_le)+list(mean)+list(std)+list(sha_entropy)+list(glcm_entropy)+list(Skewness)+list(Kurtosis)
        df1.loc[j]=({'class':c,'Area':area,
                          'PA_ratio':PA_ratio,
                          'Solidity':Solidity,
                          'Circularity':Circularity,
                          'EquivDiameter':EquivDiameter,
                          'ConvexArea':ConvexArea,
                          'Extent':Extent,
                          'FilledArea':FilledArea,
                          'Major axis length':maj_ax_le,
                          'Minor axis length':min_ax_le,
                          #'LS_ Ratio':[LS_ratio],
                          'Mean':mean,
                          'Standard Deviation':std,
                          'Shannon_Entropy':sha_entropy,
                          'Glcm_entropy':glcm_entropy,
                          'Skewness':Skewness,
                          'kurtosis':Kurtosis,
                          'lbp_energy':lbp_energy,
                          'lbp_entropy':lbp_entropy,
                          'gabor_energy':gabor_energy,
                          'gabor_entropy':gabor_entropy})

        j+=1
    else:
        print('black found \n')
        black+=1
    print('\n')
      
print('\n\n\n') 
print('black images found: ',black, 'out of total image : ',len(onlyfiles), )    
print('\n\n\n\n\n\n')    
    
    
    
    
frames = df1


############################################### give directy to save csv file

#roidata_set = pd.concat(frames)
frames.to_csv(csv,index=False)### give train.csv path
#print(frames)

###############################################################################


######## after completing train set give me an ss of the output that will be printed below... do the same for val and test also.

250
working on img  75.png


working on img  189.png


working on img  131.png


working on img  129.png


working on img  241.png


working on img  53.png


working on img  52.png


working on img  14.png


working on img  232.png


working on img  10.png


working on img  31.png


working on img  140.png


working on img  106.png


working on img  74.png


working on img  147.png


working on img  181.png


working on img  175.png


working on img  51.png


working on img  25.png


working on img  48.png


working on img  78.png


working on img  220.png


working on img  118.png


working on img  114.png


working on img  104.png


working on img  142.png


working on img  33.png


working on img  145.png


working on img  66.png


working on img  208.png


working on img  13.png


working on img  39.png


working on img  201.png


working on img  231.png


working on img  108.png


working on img  72.png


working on img  133.png


working on img  135.png


working on img  179.png
